In [1]:
import pandas as pd
import numpy as np

In [3]:
# Not stable URL from here: https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/
url = "https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/_w_6e35d5ea/session/ed5318561bde5aa8632ef36e24561cdf/download/downloadData?w=6e35d5ea"

In [4]:
nba = pd.read_csv(url)
nba.head()

<ipython-input-4-e68c0c818c74>:1: DtypeWarning: Columns (68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  nba = pd.read_csv(url)


,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537


In [7]:
# Set pd options display all 7:20 pm
pd.options.display.max_rows = None
nba.T

,0,1,2,3,4,5,6,7,8,9,...,112113,112114,112115,112116,112117,112118,112119,112120,112121,112122
game_id,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,...,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW
game_date,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,...,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07
OT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H_A,A,A,A,A,A,A,A,A,A,A,...,H,H,H,H,H,H,H,H,H,H
Team_Abbrev,WAS,WAS,WAS,WAS,WAS,WAS,WAS,WAS,WAS,WAS,...,GSW,GSW,GSW,GSW,GSW,GSW,GSW,GSW,GSW,GSW
Team_Score,117,117,117,117,117,117,117,117,117,117,...,118,118,118,118,118,118,118,118,118,118
Team_pace,94.5,94.5,94.5,94.5,94.5,94.5,94.5,94.5,94.5,94.5,...,90.9,90.9,90.9,90.9,90.9,90.9,90.9,90.9,90.9,90.9
Team_efg_pct,0.627,0.627,0.627,0.627,0.627,0.627,0.627,0.627,0.627,0.627,...,0.606,0.606,0.606,0.606,0.606,0.606,0.606,0.606,0.606,0.606
Team_tov_pct,13.5,13.5,13.5,13.5,13.5,13.5,13.5,13.5,13.5,13.5,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
Team_orb_pct,22.9,22.9,22.9,22.9,22.9,22.9,22.9,22.9,22.9,22.9,...,18.9,18.9,18.9,18.9,18.9,18.9,18.9,18.9,18.9,18.9


In [10]:
nba = nba.drop('Inactives', axis=1)

**Strategy for normalization down to the 3d NF:**

- T1: info about just the team
- T2: info about the team in each game 
- T3: info about the player in each game
- T4: inf about players, not dependent on games (season totals)
- T5: info about the teams, not dependent on games (season totals)

TODO from the book - instal mySQL and postgres